### Notebook for Clustering and Segmenting  of Toronto Neighbourhood

##### Scraping the tabular data with pandas from the specified URL

In [1]:
import pandas as pd
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_list = pd.read_html(url)

Since the page has many tables, check the table count and access the required tables with index value accordingly. Here it is [0]

In [2]:
#len(df_list)
df_list[0].head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Size of the dataframe before cleaning the data i.e ignoring the Borough = = Not Assigned

In [3]:
df_list[0].shape


(180, 3)

Delete the Borough with "Not Assigned" Value

In [4]:
df_list[0].drop(df_list[0][df_list[0].Borough == 'Not assigned'].index, inplace =True)
print(df_list[0].shape)


(103, 3)


Check for Neighbourhood with "Not Assigned" Value and assign the Borough value to neighbourhood. Since all the values are specified here no need to explicitly assign

In [5]:
df_list[0].loc[df_list[0]['Neighbourhood'] == 'Not Assigned']

# code to assign the borough value to neighbourhood if the Neighbourhood is not assigned.
#df_list[0].loc[df_list[0].Neighbourhood == 'Not assigned',"Neighbourhood"] = df_list[0].Borough

,Postal Code,Borough,Neighbourhood


To check for the duplicate postal code is any and join the neighbourhood values. In the data specified , since there is no repetition of postal code we ignore merging the neighbourhood values.

In [6]:
df1 = df_list[0].groupby(['Postal Code']).filter(lambda x: len(x) > 1)
df1

# Code to merge the neighbourhood values if the postal code is repeated..

#df_list[0].groupby('Postal Code')['Neighbourhood'].apply(','.join).reset_index()
#df_list[0].shape

,Postal Code,Borough,Neighbourhood


Size of the Dataframe

In [7]:
df_list[0].shape

(103, 3)